# ChatBot 

#### ChatBots are commonly used in customer support, virtual assistants, and other applications that require natural language processing and interaction.

## Importing Required Libraries 

In [1]:
import numpy as np
import json
import random
import pickle

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.models import load_model

from keras.optimizers import SGD

import tkinter
from tkinter import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


# Loading Json File

In [2]:
import json

file_location = "D:/User/Desktop/Chatbot/response.json"


with open(file_location, "r") as file:
    intents = json.load(file)

print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [3]:
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi',
    'Hey',
    'How are you',
    'Is anyone there?',
    'Hello',
    'Good day'],
   'responses': ['Hey :-)',
    'Hello, thanks for visiting',
    'Hi there, what can I do for you?',
    'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later, thanks for visiting',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'items',
   'patterns': ['Which items do you have?',
    'What kinds of items are there?',
    'What do you sell?'],
   'responses': ['We sell coffee and tea', 'We have coffee and tea']},
  {'tag': 'payments',
   'patterns': ['Do you take credit cards?',
    'Do you accept Mastercard?',
    'Can I pay with Paypal?',
    'Are you cash only?'],
   'responses': ['We 

# Processing

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    w = nltk.word_tokenize(pattern)
    words.extend(w)
    documents.append((w, intent['tag']))

    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [WordNetLemmatizer().lemmatize(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [5]:
print(len(words), ' Word (uniques)\n', words, '\n\n')
print(len(classes), ' Class (tags)\n', classes, '\n\n')
print(len(documents), ' Document (coubles)\n', documents)

55  Word (uniques)
 ["'s", 'a', 'accept', 'anyone', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'delivery', 'do', 'doe', 'funny', 'get', 'good', 'goodbye', 'have', 'hello', 'helpful', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'only', 'pay', 'paypal', 'see', 'sell', 'shipping', 'something', 'take', 'tell', 'thank', 'thanks', 'that', 'there', 'what', 'when', 'which', 'with', 'you'] 


7  Class (tags)
 ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks'] 


26  Document (coubles)
 [(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thank', "'s", 'a', 'lot', '!'], 'thanks'

In [6]:
output_empty = [0] *len(classes)
training = []

for doc in documents:
    bag = []
    pattern_words = doc[0]

    for word in pattern_words:
      pattern_words = WordNetLemmatizer().lemmatize(word.lower())

    for w in words:
      if w in pattern_words:
        bag.append(1)
      else:
        bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [7]:
random.shuffle(training)
training = np.array(training)
training

C:\Users\HP\AppData\Local\Temp\ipykernel_3488\1171096522.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([1, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 1, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 1, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([1, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

# Model Training

In [8]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [9]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [10]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
history = model.fit(np.array(train_x), np.array(train_y), epochs=500, batch_size=3)

Epoch 1/500
9/9 [==============================] - 1s 3ms/step - loss: 1.9868 - accuracy: 0.0385
Epoch 2/500
9/9 [==============================] - 0s 2ms/step - loss: 1.9006 - accuracy: 0.0769
Epoch 3/500
9/9 [==============================] - 0s 2ms/step - loss: 1.8870 - accuracy: 0.2308
Epoch 4/500
9/9 [==============================] - 0s 3ms/step - loss: 1.8610 - accuracy: 0.2692
Epoch 5/500
9/9 [==============================] - 0s 2ms/step - loss: 1.7907 - accuracy: 0.3077
Epoch 6/500
9/9 [==============================] - 0s 2ms/step - loss: 1.9321 - accuracy: 0.2308
Epoch 7/500
9/9 [==============================] - 0s 2ms/step - loss: 1.8019 - accuracy: 0.2692
Epoch 8/500
9/9 [==============================] - 0s 2ms/step - loss: 1.7611 - accuracy: 0.2692
Epoch 9/500
9/9 [==============================] - 0s 3ms/step - loss: 1.7596 - accuracy: 0.2692
Epoch 10/500
9/9 [==============================] - 0s 2ms/step - loss: 1.7048 - accuracy: 0.3462
Epoch 11/500
9/9 [===========

In [11]:
model.save('chatbot-model.h5', history)
print('Model \"chatbot-model.h5\" Saved')

Model "chatbot-model.h5" Saved


# Testing


In [12]:
model = load_model('chatbot-model.h5')
print('Model \"chatbot-model.h5\" Loaded')

Model "chatbot-model.h5" Loaded


In [13]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [WordNetLemmatizer().lemmatize(word.lower()) for word in sentence_words]
  return sentence_words

In [14]:
def bow(sentence, words, show_details=True):
  sentence_words = clean_up_sentence(sentence)
  bag=[0] *len(words)

  for s in sentence_words:
      for i, w in enumerate(words):
          if w == s:
              bag[i]=1
              if show_details:
                print("found in bag: %s" % w)
  return (np.array(bag))

In [15]:
def predict_class(sentence, model):
  return_list = []
  p = bow(sentence, words, show_details = False)
  res = model.predict(np.array([p]))[0]
  error_threshhold = 0.25
  results = [[i, r] for i, r in enumerate(res) if r > error_threshhold]
  results.sort(key = lambda x:x[1], reverse=True)

  for r in results:
      return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
  return return_list

In [16]:
def getResponse(ints, intents_json):
  tag = ints[0]['intent']
  list_of_intents = intents_json['intents']
  for i in list_of_intents:
      if(i['tag'] == tag):
          result = random.choice(i['responses'])
          break
  return result

In [17]:
def chatbot_response(text):
  ints = predict_class(text, model)
  res = getResponse(ints, intents)
  return res

# Graphical User Interface (GUI)

In [19]:
def send():
  msg = EntryBox.get("1.0", 'end-1c').strip()
  EntryBox.delete("0.0", END)

  if msg != '':
    ChatLog.config(state=NORMAL)
    ChatLog.insert(END, "You >> " + msg + '\n\n')
    ChatLog.config(foreground="#FFFFFF", font=("Cambria", 13, 'bold' ))

    res = chatbot_response(msg)
    ChatLog.insert(END, "Codeclause >>  " + res + '\n\n')

    ChatLog.config(state=DISABLED)
    ChatLog.yview(END)

In [21]:
base = Tk()
base.title("ChatBot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

ChatLog = Text(base, bd=0, bg="#000000",height=10, width=100, font="Arial",)
ChatLog.config(state=DISABLED)

scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(
    base, bd=1,
    font=("Cambria", 13, 'bold'),
    text="Send", command=send,
    width=10, height=3,
    bg="#DC143C", fg='#000000',
    activebackground="#FFFFFF"
    )

EntryBox = Text(
    base, bd=2,
    font="Cambria",
    width=29, height=2,
    bg="white",
    )

scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(relheight=0.100, relwidth=0.77, relx=0.01, rely=0.80)
SendButton.place(relx=0.79, rely=0.80, relheight=0.100, relwidth=0.20)

base.mainloop()

1/1 [==============================] - 0s 20ms/step
